# More on Using NER to Map Geographic Metadata in MARC Records
*Now with a **better** strategy for disambiguating placenames!*

This post follows up on a previous posts about using NER to map placenames found in ISAW Library MARC records. One problem that we encountered in the last post was the disambiguation of placename data. 

To summarize the work completed so far: using the American School of Classical Studies at Athens's 1947 [*Ancient Corinth: A guide to the excavations*](http://www.worldcat.org/title/ancient-corinth-a-guide-to-the-excavations/oclc/10220993), we can use the Stanford NER tagger to return possible location-topics for the book—in this case, Corinth (thankfully!), Athens, and Greece. Using Geonames, we can get a list of possible matches for these places, retrieve the latitude and longitude data, and map these points. That said, a query for 'athens' yields both "Athens, Greece" and "Athens, GA" so some disambiguation strategy is necessary.

In this notebook, I use geographic clustering with DBSCAN to isolate a 'region' for the coordinates that best suits the original research question: "Where is this book about?" Using a threshold of 1000km, the cities of Athens and Corinth and the country of Greece all fall into one cluster, while Athens, GA falls into another (and, for that matter, Corinth in Saint Lucia into yet another). Assuming that a book is geographically coherent—a large and problematic assumption, though perhaps less so for an archaeological site report—we work from the premise that the highest frequency locations will cluster together. From here (for now), we take the closest coordinates to the cluster's center. As shown below, this strategy works for *Ancient Corinth*. More testing is neeeded, but this experiment is encouraging and it moves the solution to our disambiguation problem further along. [PJB 6.22.18]

In [1]:
# Imports

%load_ext dotenv

import os

import json
import requests

from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.chunk import conlltags2tree
from nltk.tree import Tree

import folium

from pprint import pprint
from tqdm.notebook import tqdm

In [2]:
%dotenv .env
# Set environment variable
# Geonames requires a username to access the API but we do not want to expose personal info in code
# Run this locally by adding USERNAME to environment variables, e.g. to .env, as follows:
# > export USERNAME=<insert username here>
USERNAME = os.getenv('USERNAME')

## Get MARC Records

In [3]:
# modified marc record

marcs = ['DF261.C65 A57 1947   (OCoLC)10220993   Ancient Corinth : a guide to the excavations.   Guide to the excavations of ancient Corinth   4th ed., rev. and enl.   [Athens? : Printing Office "Hestia"], 1947.   127 p., [2] leaves of plates : ill., plans (some fold.) ; 21 cm.   At head of title: American School of Classical Studies at Athens. Corinth (Greece) Antiquities.   Excavations (Archaeology) Greece Corinth.   Greece Antiquities.   Broneer, Oscar, 1894-1992.   Carpenter, Rhys, 1889-1980. Ancient Corinth, a guide to the excavations and museums.   Morgan, Charles H. (Charles Hill), 1902-1984.   American School of Classical Studies at Athens.']
print(marcs)

['DF261.C65 A57 1947   (OCoLC)10220993   Ancient Corinth : a guide to the excavations.   Guide to the excavations of ancient Corinth   4th ed., rev. and enl.   [Athens? : Printing Office "Hestia"], 1947.   127 p., [2] leaves of plates : ill., plans (some fold.) ; 21 cm.   At head of title: American School of Classical Studies at Athens. Corinth (Greece) Antiquities.   Excavations (Archaeology) Greece Corinth.   Greece Antiquities.   Broneer, Oscar, 1894-1992.   Carpenter, Rhys, 1889-1980. Ancient Corinth, a guide to the excavations and museums.   Morgan, Charles H. (Charles Hill), 1902-1984.   American School of Classical Studies at Athens.']


## Named Entity Extraction on MARC Record

In [4]:
# Setup Stanford NER Tagger

st = StanfordNERTagger('/usr/local/share/stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz', 
                       '/usr/local/share/stanford-ner/stanford-ner.jar',
                       encoding='utf-8')

In [5]:
# Functions for putting together with inside-outside-beginning (IOB) logic
# Cf. https://stackoverflow.com/a/30666949
#
# For more information on IOB tagging, see https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)

def stanfordNE2BIO(tagged_sent):
    bio_tagged_sent = []
    prev_tag = "O"
    for token, tag in tagged_sent:
        if tag == "O": #O
            bio_tagged_sent.append((token, tag))
            prev_tag = tag
            continue
        if tag != "O" and prev_tag == "O": # Begin NE
            bio_tagged_sent.append((token, "B-"+tag))
            prev_tag = tag
        elif prev_tag != "O" and prev_tag == tag: # Inside NE
            bio_tagged_sent.append((token, "I-"+tag))
            prev_tag = tag
        elif prev_tag != "O" and prev_tag != tag: # Adjacent NE
            bio_tagged_sent.append((token, "B-"+tag))
            prev_tag = tag

    return bio_tagged_sent


def stanfordNE2tree(ne_tagged_sent):
    bio_tagged_sent = stanfordNE2BIO(ne_tagged_sent)
    sent_tokens, sent_ne_tags = zip(*bio_tagged_sent)
    sent_pos_tags = [pos for token, pos in pos_tag(sent_tokens)]

    sent_conlltags = [(token, pos, ne) for token, pos, ne in zip(sent_tokens, sent_pos_tags, sent_ne_tags)]
    ne_tree = conlltags2tree(sent_conlltags)
    return ne_tree

In [6]:
# Apply NER to each MARC record; extract locations

places_set = []

for marc in tqdm(marcs):
    marc_coordinates = []
    tokenized_marc = word_tokenize(marc)
    classified_marc = st.tag(tokenized_marc)
    classified_marc = [item for item in classified_marc if item[0] != ''] # clean up parsing
    ne_tree = stanfordNE2tree(classified_marc)

    ne_in_sent = []
    for subtree in ne_tree:
        if type(subtree) == Tree: # If subtree is a noun chunk, i.e. NE != "O"
            ne_label = subtree.label()
            ne_string = " ".join([token for token, pos in subtree.leaves()])
            ne_in_sent.append((ne_string, ne_label))
    
    locations = set([tag[0] for tag in ne_in_sent if tag[1] == 'LOCATION']) # If we don't make this a set, we can use frequency info for map weight
    
    places_set.append(locations)
    

In [7]:
places_set = [sorted(item) for item in places_set]
print(places_set[0])

['Athens', 'Corinth', 'Greece', 'Greece Antiquities', 'Greece Corinth']


## Geolocate place names

In [8]:
# Function for querying geonames

def geonames_query(location):
    '''
    queries geonames for given location name;
    bounding box variables contain default values
    based on: https://prpole.github.io/location-extraction-georeferencing/    
    '''
    # Todo
    # - replace error handling

    baseurl = 'http://api.geonames.org/searchJSON' #baseurl for geonames
    username = USERNAME # Keep USERNAME in .env
    json_decode = json.JSONDecoder() #used to parse json response

    params = {
        'username': username, 
        'name_equals': location,
        'orderby': 'relevance',
    }
    
    response = requests.get(baseurl, params=params)
    response_string = response.text
    parsed_response = json_decode.decode(response_string)
    
    if 'geonames' in parsed_response.keys():
        if len(parsed_response['geonames']) > 0:
            first_response = parsed_response['geonames'][0]
            coordinates = (first_response['lat'],first_response['lng'])
        else: 
            coordinates = ('','')
    else:
        coordinates = ('','')
    return coordinates

In [9]:
# Build list of likely coordinates for places

places_list = []
coordinates_list = []

for places in places_set[:1]:
    coordinates = []
    for place in places:
        print(place)
        ll = geonames_query(place)
        if ll != ('',''):
            places_list.append(place)
            coordinates.append(ll)
    coordinates_list.append(coordinates)    

print(coordinates_list[0])

Athens
Corinth
Greece
Greece Antiquities
Greece Corinth
[('37.98376', '23.72784'), ('33.15401', '-97.06473'), ('39', '22')]


In [10]:
# Convert coordinates to float

coordinates_list = [[(float(lat), float(long)) for lat, long in item] for item in coordinates_list]

In [11]:
# Get sample

print(places_list)
print(coordinates_list[0])

['Athens', 'Corinth', 'Greece']
[(37.98376, 23.72784), (33.15401, -97.06473), (39.0, 22.0)]


## Create map

In [12]:
# Set up Folium and populate with coordinates

basemap = folium.Map(location=[37.97945, 23.71622], zoom_start=4, tiles='cartodbpositron', width=960, height=512)

for i, c in enumerate(coordinates_list[0]):
    folium.Marker([c[0], c[1]], popup='{} {}{}'.format(places_list[i], c[0], c[1])).add_to(basemap)
    
basemap

In [13]:
# Why does only Athens return a result on the map? Check zoom...

basemap.zoom_start = 2
basemap

## Disambiguate placenames

In [14]:
# So we have learned that we can't use the top 'relevance' score from the Geoname API, or we
# get results like 'Corinth, Texas' and 'Greece, NY'. We need some plan for disambiguating the
# Geonames results...

In [15]:
import geocoder

In [16]:
places_list

['Athens', 'Corinth', 'Greece']

In [17]:
# Retrieve json from geonames API (for fun this time using geocoder)

geocoder_results = []

for place in places_list:
    results = geocoder.geonames(place, maxRows=10, key=USERNAME)
    jsons = []
    for result in results:
        jsons.append(result.json)
    geocoder_results.append(jsons)
    
pprint(geocoder_results[1][0]) # Corinth, but a wrong Corinth

{'address': 'Corinth',
 'class_description': 'country, state, region,...',
 'code': 'ADM2',
 'country': 'Saint Lucia',
 'country_code': 'LC',
 'description': 'second-order administrative division',
 'feature_class': 'A',
 'geonames_id': 11351423,
 'lat': '14.0471',
 'lng': '-60.96046',
 'ok': True,
 'population': 1382,
 'raw': {'adminCode1': '06',
         'adminCodes1': {'ISO3166_2': '06'},
         'adminName1': 'Gros-Islet',
         'countryCode': 'LC',
         'countryId': '3576468',
         'countryName': 'Saint Lucia',
         'fcl': 'A',
         'fclName': 'country, state, region,...',
         'fcode': 'ADM2',
         'fcodeName': 'second-order administrative division',
         'geonameId': 11351423,
         'lat': '14.0471',
         'lng': '-60.96046',
         'name': 'Corinth',
         'population': 1382,
         'toponymName': 'Corinth'},
 'state': 'Gros-Islet',
 'state_code': '06',
 'status': 'OK'}


In [18]:
# Iterate over geocoder_results and keep all lat/long

coordinates = []

for i, results in enumerate(geocoder_results):
    for item in results:
        if item['lat'] and item['lng']:
            coordinates.append((item['address'], float(item['lat']), float(item['lng'])))

print(coordinates)

[('Athens', 37.98376, 23.72784), ('Athens', 33.96095, -83.37794), ('Athens', 39.32924, -82.10126), ('Athens', 39.33386, -82.04513), ('Athens Airport', 37.93636, 23.94447), ('The Plains', 39.36896, -82.13237), ('Athens', 13.90574, -60.89184), ('Strouds Run State Park', 39.34924, -82.03236), ('Chauncey', 39.39785, -82.12931), ('Glouster', 39.50313, -82.08459), ('Corinth', 14.0471, -60.96046), ('Corinth Estate', 14.04336, -60.95388), ('Corinth Head', -53.0, 73.41667), ('Corinth', 37.94007, 22.9513), ('Corinth/La Bel Lair', 14.04469, -60.94484), ('Corinth', 37.8144, 22.94352), ('Achaea (Roman province)', 37.93445, 22.92615), ('Ancient Corinth', 37.90953, 22.88353), ('Corinth', 34.93425, -88.52227), ('Examilia', 37.89736, 22.92832), ('Greece', 39.0, 22.0), ('Pátrai', 38.24444, 21.73444), ('Central Greece', 38.35243, 23.13995), ('West Greece', 38.48799, 21.2915), ('Central Greece and Euboea', 38.66667, 22.5), ('Lamia', 38.9, 22.43333), ('Achaea', 38.13333, 22.0), ('Peloponnese', 37.5, 22.333

## Create revised map

Still not what we are looking for, but the 'cluster' premise gives us a reason to believe that one subset of coordinates, i.e. the one near Greece, are a better fit that the others.

In [19]:
# Set up Folium and populate with all coordinates

basemap = folium.Map(location=[30, 0], zoom_start=2, tiles='cartodbpositron', width=960, height=512)

for i, c in enumerate(coordinates):
    folium.Marker([c[1], c[2]]).add_to(basemap)

print('Still not what we are looking for...')
basemap

Still not what we are looking for...


## Cluster the coordinates with DBSCAN

Following this [notebook](https://github.com/gboeing/2014-summer-travels/blob/master/clustering-scikitlearn.ipynb) by Geoff Boeing for geographical clustering with DBScan

In [20]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from sklearn.cluster import DBSCAN
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
%matplotlib inline

# define the number of kilometers in one radian
kms_per_radian = 6371.0088

In [21]:
# make dataframe

df = pd.DataFrame.from_records(coordinates, columns=['placename', 'lat', 'lon'])
df[:5]

,placename,lat,lon
0,Athens,37.98376,23.72784
1,Athens,33.96095,-83.37794
2,Athens,39.32924,-82.10126
3,Athens,39.33386,-82.04513
4,Athens Airport,37.93636,23.94447


In [22]:
# represent points consistently as (lat, lon)
coords = df[['lat', 'lon']].values

# define epsilon as 1.5 kilometers, converted to radians for use by haversine
epsilon = 1000 / kms_per_radian

In [23]:
# run the clustering algorithm

start_time = time.time()
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_

# get the number of clusters
num_clusters = len(set(cluster_labels))

# all done, print the outcome
message = 'Clustered {:,} points down to {:,} clusters, for {:.1f}% compression in {:,.2f} seconds'
print(message.format(len(df), num_clusters, 100*(1 - float(num_clusters) / len(df)), time.time()-start_time))
print('Silhouette coefficient: {:0.03f}'.format(metrics.silhouette_score(coords, cluster_labels)))

Clustered 30 points down to 4 clusters, for 86.7% compression in 0.01 seconds
Silhouette coefficient: 0.934


In [24]:
# turn the clusters in to a pandas series, where each element is a cluster of points

clusters = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])
print(clusters)

0    [[37.98376, 23.72784], [37.93636, 23.94447], [...
1    [[33.96095, -83.37794], [39.32924, -82.10126],...
2    [[13.90574, -60.89184], [14.0471, -60.96046], ...
3                                  [[-53.0, 73.41667]]
dtype: object


In [25]:
# get centers

def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

centermost_points = clusters.map(get_centermost_point)

In [26]:
# summarize clusters

max_cluster_index = 0
max_temp = 0

for i, cluster in enumerate(clusters):
    print(f'Cluster {i} centered at {centermost_points[i]} contains {len(cluster)} points.')
    if len(cluster) > max_temp:
        max_temp = len(cluster)
        max_cluster_index = i

Cluster 0 centered at (37.93445, 22.92615) contains 17 points.
Cluster 1 centered at (39.32924, -82.10126) contains 8 points.
Cluster 2 centered at (14.04469, -60.94484) contains 4 points.
Cluster 3 centered at (-53.0, 73.41667) contains 1 points.


In [27]:
# keep only largest cluster

main_cluster = clusters[max_cluster_index]
main_center = centermost_points[max_cluster_index]

In [28]:
# convert main_cluster coordinates from numpy array to tuple of floats

main_cluster_coordinates = tuple(map(tuple, main_cluster))

In [29]:
# make new map

basemap = folium.Map(location=main_center, zoom_start=6, tiles='cartodbpositron', width=960, height=512)

for i, c in enumerate(main_cluster_coordinates):
    folium.Marker([c[0], c[1]]).add_to(basemap)

print("All points in largest cluster...")
basemap

All points in largest cluster...


In [30]:
# define functions to compute distance from center following https://stackoverflow.com/a/41337005

from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v[0],v[1],p[0],p[1]))

In [31]:
# iterate over results and find closest distance from main cluster center

top_places = []
top_coordinates = []

for results in geocoder_results:

    coords_ = []
    for result in results:
        coords_.append((float(result['lat']), float(result['lng'])))
    closest_coordinates = closest(coords_, main_center)
    top_places.append(next(item['address'] for item in results if float(item['lat']) == closest_coordinates[0]))
    top_coordinates.append(closest_coordinates)

In [32]:
# set up Folium and populate with coordinates

basemap = folium.Map(location=main_center, zoom_start=9, tiles='cartodbpositron', width=960, height=512)

for i, c in enumerate(top_coordinates):
    folium.Marker([c[0], c[1]], popup='{} {}{}'.format(top_places[i], c[0], c[1])).add_to(basemap)
    
print('Map of relevant locations in Broneer et al.\'s "Ancient Corinth: A guide to the excavations"')
basemap

Map of relevant locations in Broneer et al.'s "Ancient Corinth: A guide to the excavations"
